In [3]:
import torch
import torch.nn as nn
from torch.distributions.categorical import Categorical
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import numpy as np
import wandb
import os
import random
from time import time

from models.LSTM import LSTM_Model
from models.data import EpisodeDataset

In [6]:
TRAIN_PATH = '/home/bdemoss/Documents/Episode_Data/30_TPS/Large_Obs_Nexto_Train/'
TEST_PATH = '/home/bdemoss/Documents/Episode_Data/30_TPS/Large_Obs_Nexto_Test/'

train_dataset = EpisodeDataset(TRAIN_PATH)
test_dataset = EpisodeDataset(TEST_PATH)

Loading Episode Data
Finished Loading Episode Data
Loading Episode Data
Finished Loading Episode Data


In [3]:
Train_Data_Loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=8)
Test_Data_Loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)

In [8]:
MODEL_PATH = 'Trained Networks/120_TPS_Nexto_LSTM_5L'

model = LSTM_Model(Train_Data_Loader, Test_Data_Loader)
model.load(MODEL_PATH)

104448
1048576
  2048
  2048
524288
  1024
1048576
  1024
1048576
  1024
1048576
  1024
1048576
  1024
 92160
    90
  2048
     2
  2048
     2
  2048
     2
______
5979232
None


In [10]:
wandb.init(project="Inverse Dynamics Model", entity="harrymead")

wandb.config = {
  "learning_rate": 0.001,
  "epochs": 3000,
  "batch_size": 10
}

accuracy/action,▁
accuracy/has_flip,▁
accuracy/has_jump,▁
accuracy/on_ground,▁
loss,▁
accuracy/action,0.73421
accuracy/has_flip,0.97158
accuracy/has_jump,0.97092
accuracy/on_ground,0.9966
loss,15.54744


In [11]:
saved_best_loss = 1000000

for epoch in range(20):
    start_time = time()
    loss = model.train(log_results=True)

    
    print('Epoch: ', epoch, 'Loss: ', loss)
    if loss <= saved_best_loss:
        model.save(MODEL_PATH)
        saved_best_loss = loss

Batch:  1000 , Loss:  72.33035178445608 , Rate:  11027.078491571285 , Time:  55.97665882110596
Action Accuracy:  0.2057807176566077 On Ground Accuracy:  0.7601817065445785 Has Jump Accuracy:  0.7779975666616444 Has Flip Accuracy:  0.8137864332476319
Batch:  2000 , Loss:  63.372165378456835 , Rate:  10783.271282594771 , Time:  112.45820641517639
Action Accuracy:  0.30732009273671834 On Ground Accuracy:  0.8284419968606125 Has Jump Accuracy:  0.8333809495661997 Has Flip Accuracy:  0.8518002219901615
Batch:  3000 , Loss:  58.2178963289887 , Rate:  10944.566121061274 , Time:  167.627671957016
Action Accuracy:  0.33151263009360776 On Ground Accuracy:  0.9342261293549476 Has Jump Accuracy:  0.8817680143624668 Has Flip Accuracy:  0.8764417121014383
Batch:  4000 , Loss:  55.95045294353747 , Rate:  11384.409459597422 , Time:  222.43631625175476
Action Accuracy:  0.3413163192040413 On Ground Accuracy:  0.9473020360540554 Has Jump Accuracy:  0.8975033335897633 Has Flip Accuracy:  0.88964535733517

KeyboardInterrupt: 

In [12]:
model.test()

Batch:  0 , Loss:  56.166204693960196 , Rate:  25637.36775889931 , Time:  0.4178285598754883
Action Accuracy:  0.30610532 On Ground Accuracy:  0.9706871 Has Jump Accuracy:  0.9115011 Has Flip Accuracy:  0.8730396


56.166204693960196